In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("train.csv")
data = data[~data.isna()["target"]]
data.drop(columns=['row_id', 'time_id'], inplace=True)

### Full Train Test

In [28]:
df = data.copy()
df['far_price'] = df['far_price'].fillna(0)
df['near_price'] = df['near_price'].fillna(0)
df.dropna(inplace=True)

X = df.drop(columns=['target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [29]:
reg = LinearRegression().fit(X_train, y_train)
#reg = RandomForestRegressor(max_depth=1, random_state=0).fit(X_train, y_train)

In [30]:
pred = reg.predict(X_test)
print("MAE:", abs(pred-y_test).mean())

MAE: 6.313735561649197


### Sequenced Before-After Train Test

In [34]:
X = data.drop(columns=['target'])
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_train_bef = X_train[X_train['near_price'].isna()]
X_train_aft = X_train[~X_train['near_price'].isna()]

X_train_bef['far_price'] = X_train_bef['far_price'].fillna(0)
X_train_bef['near_price'] = X_train_bef['near_price'].fillna(0)

X_train_bef.dropna(inplace=True)
X_train_aft.dropna(inplace=True)

y_train_bef = y_train[y_train.index.isin(X_train_bef.index)]
y_train_aft = y_train[y_train.index.isin(X_train_aft.index)]

In [35]:
X_test['far_price'] = X_test['far_price'].fillna(0)
X_test['near_price'] = X_test['near_price'].fillna(0)
X_test.dropna(inplace=True)

y_test = y_test[y_test.index.isin(X_test.index)]

In [36]:
reg_bef = LinearRegression().fit(X_train_bef, y_train_bef)
reg_aft = LinearRegression().fit(X_train_aft, y_train_aft)

In [37]:
X_test_bef, X_test_aft = X_test[X_test['near_price'] == 0], X_test[X_test['near_price'] != 0]

pred_bef = reg_bef.predict(X_test_bef)
pred_aft = reg_aft.predict(X_test_aft)
diff_bef = abs(pred_bef - y_test[y_test.index.isin(X_test_bef.index)])
diff_aft = abs(pred_aft - y_test[y_test.index.isin(X_test_aft.index)])
print("MAE:", (sum(diff_bef) + sum(diff_aft)) / len(X_test))

MAE: 6.309181722683357
